In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Set display options
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)

# ============================================
# 1. LOAD DATASETS (SAMPLE FIRST)
# ============================================

# Load samples to understand structure
print("Loading data samples...")

# Flight Data - sample 100k rows
flight_df = pd.read_csv('../data/raw/flight_data_2018_2024.csv', nrows=100000)
print(f"Flight Data Shape: {flight_df.shape}")

# POI Data - sample 50k rows (large file)
poi_df = pd.read_csv('../data/raw/poi_data_osm.csv', nrows=50000)
print(f"POI Data Shape: {poi_df.shape}")

# Attractions Data - load all (small)
attractions_df = pd.read_csv('../data/raw/attractions_data_USA.csv')
print(f"Attractions Data Shape: {attractions_df.shape}")



In [1]:
# ============================================
# 2. DATA PROFILING
# ============================================

def profile_dataset(df, name):
    """Generate comprehensive data profile"""
    print(f"\n{'='*60}")
    print(f"PROFILING: {name}")
    print(f"{'='*60}\n")
    
    # Basic info
    print(f"Shape: {df.shape}")
    print(f"Memory Usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB\n")
    
    # Column types
    print("Data Types:")
    print(df.dtypes.value_counts())
    print()
    
    # Missing values
    print("Missing Values:")
    missing = df.isnull().sum()
    missing_pct = (missing / len(df)) * 100
    missing_df = pd.DataFrame({
        'Missing_Count': missing,
        'Percentage': missing_pct
    }).sort_values('Percentage', ascending=False)
    print(missing_df[missing_df['Missing_Count'] > 0].head(20))
    print()
    
    # Duplicates
    print(f"Duplicate Rows: {df.duplicated().sum()}")
    print()
    
    # Numeric columns summary
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    if len(numeric_cols) > 0:
        print("Numeric Columns Summary:")
        print(df[numeric_cols].describe())
    
    return missing_df

# Profile each dataset
flight_profile = profile_dataset(flight_df, "FLIGHT DATA")
poi_profile = profile_dataset(poi_df, "POI DATA")
attractions_profile = profile_dataset(attractions_df, "ATTRACTIONS DATA")



NameError: name 'flight_df' is not defined

In [ ]:
# ============================================
# 3. FLIGHT DATA SPECIFIC ANALYSIS
# ============================================

print("\n" + "="*60)
print("FLIGHT DATA DEEP DIVE")
print("="*60 + "\n")

# Key columns for flights
flight_key_cols = ['FlightDate', 'Marketing_Airline_Network', 'Origin', 'Dest', 
                   'DepDelay', 'ArrDelay', 'Cancelled', 'Distance']

print("Available Key Columns:")
available_cols = [col for col in flight_key_cols if col in flight_df.columns]
print(available_cols)
print()

# Date range
if 'FlightDate' in flight_df.columns:
    flight_df['FlightDate'] = pd.to_datetime(flight_df['FlightDate'], errors='coerce')
    print(f"Date Range: {flight_df['FlightDate'].min()} to {flight_df['FlightDate'].max()}")
    print()

# Top airlines
if 'Marketing_Airline_Network' in flight_df.columns:
    print("Top 10 Airlines:")
    print(flight_df['Marketing_Airline_Network'].value_counts().head(10))
    print()

# Top routes
if 'Origin' in flight_df.columns and 'Dest' in flight_df.columns:
    flight_df['Route'] = flight_df['Origin'] + '-' + flight_df['Dest']
    print("Top 10 Routes:")
    print(flight_df['Route'].value_counts().head(10))
    print()

# Delay statistics
if 'DepDelay' in flight_df.columns:
    print("Departure Delay Statistics:")
    print(flight_df['DepDelay'].describe())
    print(f"Flights with delays > 15 min: {(flight_df['DepDelay'] > 15).sum()}")
    print()

# Cancellation rate
if 'Cancelled' in flight_df.columns:
    cancellation_rate = (flight_df['Cancelled'].sum() / len(flight_df)) * 100
    print(f"Cancellation Rate: {cancellation_rate:.2f}%")
    print()



In [ ]:
# ============================================
# 4. POI DATA SPECIFIC ANALYSIS
# ============================================

print("\n" + "="*60)
print("POI DATA DEEP DIVE")
print("="*60 + "\n")

# Key columns for POI
poi_key_cols = ['latitude', 'longitude', 'name', 'amenity', 'tourism', 
                'addr:city', 'addr:state']

print("Available Key Columns:")
available_poi_cols = [col for col in poi_key_cols if col in poi_df.columns]
print(available_poi_cols)
print()

# Coordinate validation
if 'latitude' in poi_df.columns and 'longitude' in poi_df.columns:
    print("Coordinate Statistics:")
    print(f"Latitude range: {poi_df['latitude'].min():.4f} to {poi_df['latitude'].max():.4f}")
    print(f"Longitude range: {poi_df['longitude'].min():.4f} to {poi_df['longitude'].max():.4f}")
    
    # Check for invalid coordinates
    invalid_lat = ((poi_df['latitude'] < -90) | (poi_df['latitude'] > 90)).sum()
    invalid_lon = ((poi_df['longitude'] < -180) | (poi_df['longitude'] > 180)).sum()
    print(f"Invalid latitude: {invalid_lat}")
    print(f"Invalid longitude: {invalid_lon}")
    print()

# Amenity types
if 'amenity' in poi_df.columns:
    print("Top 15 Amenity Types:")
    print(poi_df['amenity'].value_counts().head(15))
    print()

# Tourism types
if 'tourism' in poi_df.columns:
    print("Top 10 Tourism Types:")
    print(poi_df['tourism'].value_counts().head(10))
    print()

# State distribution
state_col = 'addr:state' if 'addr:state' in poi_df.columns else 'state'
if state_col in poi_df.columns:
    print("Top 10 States by POI Count:")
    print(poi_df[state_col].value_counts().head(10))
    print()



In [ ]:
# ============================================
# 5. ATTRACTIONS DATA SPECIFIC ANALYSIS
# ============================================

print("\n" + "="*60)
print("ATTRACTIONS DATA DEEP DIVE")
print("="*60 + "\n")

# Category distribution
if 'main_category' in attractions_df.columns:
    print("Main Categories:")
    print(attractions_df['main_category'].value_counts())
    print()

# Rating statistics
if 'rating' in attractions_df.columns:
    print("Rating Statistics:")
    print(attractions_df['rating'].describe())
    print()

# Reviews statistics
if 'reviews' in attractions_df.columns:
    print("Reviews Statistics:")
    print(attractions_df['reviews'].describe())
    print()

# State distribution
if 'state' in attractions_df.columns:
    print("Top 10 States:")
    print(attractions_df['state'].value_counts().head(10))
    print()



In [ ]:
# ============================================
# 6. DATA QUALITY VISUALIZATIONS
# ============================================

fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 1. Missing data heatmap for flight data
flight_missing = flight_df.isnull().sum().sort_values(ascending=False).head(20)
axes[0, 0].barh(range(len(flight_missing)), flight_missing.values)
axes[0, 0].set_yticks(range(len(flight_missing)))
axes[0, 0].set_yticklabels(flight_missing.index, fontsize=8)
axes[0, 0].set_xlabel('Missing Count')
axes[0, 0].set_title('Flight Data - Top 20 Columns with Missing Values')
axes[0, 0].invert_yaxis()

# 2. Flight delays distribution
if 'DepDelay' in flight_df.columns:
    delay_data = flight_df['DepDelay'].dropna()
    delay_data = delay_data[(delay_data >= -30) & (delay_data <= 120)]  # Filter outliers
    axes[0, 1].hist(delay_data, bins=50, edgecolor='black')
    axes[0, 1].set_xlabel('Departure Delay (minutes)')
    axes[0, 1].set_ylabel('Frequency')
    axes[0, 1].set_title('Distribution of Departure Delays')
    axes[0, 1].axvline(0, color='red', linestyle='--', label='On-time')
    axes[0, 1].legend()

# 3. POI coordinate distribution
if 'latitude' in poi_df.columns and 'longitude' in poi_df.columns:
    valid_coords = poi_df[(poi_df['latitude'].notna()) & (poi_df['longitude'].notna())]
    axes[1, 0].scatter(valid_coords['longitude'], valid_coords['latitude'], 
                       alpha=0.1, s=1)
    axes[1, 0].set_xlabel('Longitude')
    axes[1, 0].set_ylabel('Latitude')
    axes[1, 0].set_title('POI Geographic Distribution')

# 4. Attractions rating distribution
if 'rating' in attractions_df.columns:
    rating_data = attractions_df['rating'].dropna()
    axes[1, 1].hist(rating_data, bins=20, edgecolor='black')
    axes[1, 1].set_xlabel('Rating')
    axes[1, 1].set_ylabel('Frequency')
    axes[1, 1].set_title('Distribution of Attraction Ratings')

plt.tight_layout()
plt.savefig('../logs/data_quality_overview.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nVisualization saved to logs/data_quality_overview.png")

